## Create temporaray csv file for the entire year of coverage

This notebook should be the first one running. It takes all the files in `input_data` and combines them in a csv that spans through 1 year of data. The data used as source should have the same structure in order to be combined.

In [2]:
import os
import pandas as pd

## NYT Data

In [5]:
# Define the path to the input-data folder
input_data_folder = '../input-data/nyt'

# List all files in the input-data folder
files = os.listdir(input_data_folder)

# Initialize an empty list to store dataframes
dataframes = []

# Loop through each file and read it into a dataframe
for file in files:
    file_path = os.path.join(input_data_folder, file)
    df = pd.read_json(file_path)
    dataframes.append(df)

# Concatenate all dataframes into one unique dataframe
full_year = pd.concat(dataframes, ignore_index=True)

# Display the unique dataframe
full_year

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name
0,Wrestling with age and a case of idea theft.,https://www.nytimes.com/2024/09/01/business/he...,Wrestling with age and a case of idea theft.,"Send questions about the office, money, career...",BU,3.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Help! I’m ‘Older’ and on the Job Hun...,"[{'name': 'subject', 'value': 'Careers and Pro...",2024-09-01T04:01:07+0000,article,SundayBusiness,Business Day,"{'original': 'By Anna Holmes', 'person': [{'fi...",News,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,1280,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,NaN
1,"Grueling shifts, abuse from the public and sub...",https://www.nytimes.com/2024/09/01/world/asia/...,"Grueling shifts, abuse from the public and sub...",Exhausted doctors resting in crowded on-call r...,A,4.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Worked to the Bone, India’s Doctors ...","[{'name': 'glocations', 'value': 'India', 'ran...",2024-09-01T04:01:25+0000,article,Foreign,World,{'original': 'By Anupreeta Das and Pragati K.B...,News,nyt://article/aeabc262-aeb0-5423-a7ac-8bb664cb...,1310,nyt://article/aeabc262-aeb0-5423-a7ac-8bb664cb...,Asia Pacific
2,"About a quarter of the residents of Maracaibo,...",https://www.nytimes.com/2024/09/01/world/ameri...,"About a quarter of the residents of Maracaibo,...",It was once a thriving metropolis in the heart...,A,1.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'What Happens When Half a Million Peo...,"[{'name': 'glocations', 'value': 'Maracaibo (V...",2024-09-01T04:01:27+0000,article,Foreign,World,{'original': 'By Frances Robles and Marian Car...,News,nyt://article/42c0d0f2-ea62-5d2b-8eba-baa04180...,1460,nyt://article/42c0d0f2-ea62-5d2b-8eba-baa04180...,Americas
3,Israel and Hamas agreed to pause the war to pe...,https://www.nytimes.com/2024/09/01/world/middl...,Israel and Hamas agreed to pause the war to pe...,Health workers on Sunday began a polio vaccina...,A,1.0,The New York Times,[],"{'main': 'In Race Against Polio, Gaza Begins V...","[{'name': 'subject', 'value': 'Israel-Gaza War...",2024-09-01T04:01:43+0000,article,Foreign,World,"{'original': 'By Bilal Shbair, Erika Solomon a...",News,nyt://article/6393c6c3-0e1f-5494-925d-165e7aaf...,1473,nyt://article/6393c6c3-0e1f-5494-925d-165e7aaf...,Middle East
4,The prospect of a strong performance by the fa...,https://www.nytimes.com/2024/09/01/world/europ...,The prospect of a strong performance by the fa...,"On Sunday, voters in the eastern German states...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'What to Watch For in East German Sta...,"[{'name': 'glocations', 'value': 'Germany', 'r...",2024-09-01T04:01:43+0000,article,Foreign,World,"{'original': 'By Christopher F. Schuetze', 'pe...",News,nyt://article/fe046102-78e5-530d-89e0-59ff09c0...,812,nyt://article/fe046102-78e5-530d-89e0-59ff09c0...,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48691,The first Black officer to achieve the rank of...,https://www.nytimes.com/2024/08/31/us/arthur-j...,The first Black officer to achieve the rank of...,"Arthur J. Gregg, the first African American Ar...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Arthur J. Gregg, Trailblazing Army O...","[{'name': 'subject', 'value': 'Deaths (Obituar...",2024-08-31T14:57:12+0000,article,Obits,U.S.,"{'original': 'By Trip Gabriel', 'person': [{'f...",Obituary (Obit),nyt://article/15ef03c9-295b-50e0-a0f4-64f9a182...,1019,nyt://article/15ef03c9-295b-50e0-a0f4-64f9a182...,NaN
48692,"In a reversal, two senior U.S. military offici...",https://www.nytimes.com/2024/08/31/world/europ...,"In a reversal, two senio

In [6]:
full_year.to_csv("../input-data/temp-data.csv")

## Zeit Data

In [3]:
# Define the path to the input-data folder
input_data_folder = '../input-data/zeit'

# List all files in the input-data folder
files = os.listdir(input_data_folder)

# Initialize an empty list to store dataframes
dataframes = []

# Loop through each file and read it into a dataframe
for file in files:
    file_path = os.path.join(input_data_folder, file)
    df = pd.read_json(file_path)
    dataframes.append(df)

# Concatenate all dataframes into one unique dataframe
full_year = pd.concat(dataframes, ignore_index=True)

# Display the unique dataframe
full_year

,uri,lang,isDuplicate,date,time,dateTime,dateTimePub,dataType,sim,url,...,body,source,authors,image,eventUri,sentiment,wgt,relevance,keywords,links
0,2024-12-590516688,deu,False,2024-12-31,23:48:05,2024-12-31 23:48:05+00:00,2024-12-31T23:35:24Z,news,0.000000,https://www.zeit.de/news/2025-01/01/feuerwehr-...,...,Kurz vor dem Jahreswechsel ist in Hamburg ein ...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/feuerwehr-...,None,NaN,473384885,1,"[News, Silvesternacht, Billstedt, Hamburg, Feu...","[{'anchorText': 'Feuerwehr', 'url': 'https://w..."
1,8481124955,deu,False,2024-12-31,23:26:31,2024-12-31 23:26:31+00:00,2024-12-31T23:25:31Z,news,0.968627,https://www.zeit.de/news/2025-01/01/zweite-squ...,...,"Das Drama um scheinbar harmlose, jedoch am End...","{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/zweite-squ...,eng-10211662,NaN,473383591,1,"[News, Netflix-Welthit, Netflix, Squid, Kinder...","[{'anchorText': 'Netflix', 'url': 'https://www..."
2,8481124174,deu,False,2024-12-31,23:25:05,2024-12-31 23:25:05+00:00,2024-12-31T23:24:01Z,news,0.000000,https://www.zeit.de/gesellschaft/zeitgeschehen...,...,Vor der Silvesternacht in Berlin hat die Poliz...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/gesellschaft/zeitgeschehen...,None,NaN,473383505,1,"[Gesellschaft, Jahreswechsel, Silvester, Feuer...","[{'anchorText': 'Polizei', 'url': 'https://www..."
3,8481124175,deu,False,2024-12-31,23:24:34,2024-12-31 23:24:34+00:00,2024-12-31T23:24:01Z,news,0.000000,https://www.zeit.de/news/2025-01/01/haus-in-hu...,...,Ein Einfamilienhaus ist in der Silvesternacht ...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/haus-in-hu...,None,NaN,473383474,1,"[News, Großeinsatz der Feuerwehr, Hude, Landkr...","[{'anchorText': 'Feuerwehr', 'url': 'https://w..."
4,8481116703,deu,False,2024-12-31,23:14:35,2024-12-31 23:14:35+00:00,2024-12-31T23:12:42Z,news,0.945098,https://www.zeit.de/news/2025-01/01/polen-uebe...,...,Polen hat zum Jahreswechsel den alle sechs Mon...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2025-01/01/polen-uebe...,eng-10209752,NaN,473382875,1,"[News, Nach sechs Monaten Ungarn, Donald Tusk,...","[{'anchorText': 'Polen', 'url': 'https://www.z..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39761,8105574979,deu,False,2024-05-01,02:23:35,2024-05-01 02:23:35+00:00,2024-05-01T02:22:46Z,news,0.800000,https://www.zeit.de/news/2024-05/01/jahrestag-...,...,Die Zusammenfassung für diesen Artikel kann le...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2024-05/01/jahrestag-...,deu-1781817,NaN,452226215,1,"[News, EU, Annalena Baerbock, Radosław Sikorsk...","[{'anchorText': 'Annalena Baerbock', 'url': 'h..."
39762,8105574980,deu,False,2024-05-01,02:23:18,2024-05-01 02:23:18+00:00,2024-05-01T02:22:46Z,news,0.588235,https://www.zeit.de/politik/ausland/2024-05/us...,...,Die Audioversion dieses Artikels wurde künstli...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/politik/ausland/2024-05/mi...,deu-1781792,NaN,452226198,1,"[Politik, US-Repräsentantenhaus, US-Repräsenta...",None
39763,8105568694,deu,False,2024-05-01,02:15:24,2024-05-01 02:15:24+00:00,2024-05-01T02:14:45Z,news,0.968627,https://www.zeit.de/news/2024-05/01/sek-einsat...,...,Die Zusammenfassung für diesen Artikel kann le...,"{'uri': 'zeit.de', 'dataType': 'news', 'title'...",[],https://img.zeit.de/news/2024-05/01/sek-einsat...,deu-1781828,NaN,452225724,1,"[News, Extremismus, Velbert, Düsseldorf, Poliz...","[{'anchorText': 'Polizei', 'url': 'https://www..."
39764,8105478890,deu,False,2024-05-01,00:13:52,2024-05-01 00:13:52+00:00,2024-05-01T00:13:17Z,news,0.686275,https://www.zeit.de/politik/ausland/2024-05/ge...,...,Die Audioversion dieses Artikels wurde künstli...,"{'uri': 'zeit.de', 'dataType': 'news

In [7]:
full_year.to_csv("../input-data/zeit-temp-data.csv")